In [18]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import display

# Load the CSV files
shark_data = pd.read_csv('shark_data.csv')
bear_data = pd.read_csv('bear_data.csv')

In [19]:
# Check the column names in shark_data
print(shark_data.columns)

# Check the column names in bear_data
print(bear_data.columns)


Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')
Index(['Date', 'Location', 'Details', 'Bear', 'Latitude', 'Longitude', 'Name',
       'Age', 'Gender'],
      dtype='object')


In [20]:
# Load the bear data
bear_data = pd.read_csv('bear_data.csv')

# Initialize the map centered around a central location (average of latitudes and longitudes)
map_center = [bear_data['Latitude'].mean(), bear_data['Longitude'].mean()]
incident_map = folium.Map(location=map_center, zoom_start=5, control_scale=True)

# Create a dictionary to hold LayerGroups for each bear species
layers = {}

# Define a function to create markers with bold titles in the popup
def create_marker(row, layer_group):
    popup_message = (
        f"<strong>Bear Species:</strong> {row['Bear']}<br>"
        f"<strong>Location:</strong> {row['Location']}<br>"
        f"<strong>Date:</strong> {row['Date']}<br>"
        f"<strong>Details:</strong> {row['Details']}<br>"
        f"<strong>Age:</strong> {row['Age']}<br>"
        f"<strong>Gender:</strong> {row['Gender']}"
    )
    marker = folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_message, max_width=300),
        icon=folium.Icon(color='green', icon='info-sign')
    )
    marker.add_to(layer_group)

# Iterate over each unique bear species to create separate layers
for species in bear_data['Bear'].unique():
    layer = folium.FeatureGroup(name=f"{species} Bear Attacks")
    filtered_data = bear_data[bear_data['Bear'] == species]
    filtered_data.apply(create_marker, axis=1, layer_group=layer)
    layers[species] = layer
    layer.add_to(incident_map)

# Add the heatmap layer
heat_data = [[row['Latitude'], row['Longitude']] for _, row in bear_data.iterrows()]
heatmap_layer = folium.FeatureGroup(name='Heat Map')
HeatMap(heat_data).add_to(heatmap_layer)
heatmap_layer.add_to(incident_map)

# Style the map title with a modern look
map_title = folium.map.CustomPane('title', z_index=650)
title_html = '''
     <div style="position: fixed; 
                 bottom: 20px; left: 50%; transform: translateX(-50%);
                 background-color: rgba(0, 0, 0, 0.7); 
                 color: white; font-size: 20px; font-family: 'Arial', sans-serif;
                 z-index: 9999; padding: 10px 20px; border-radius: 10px;
                 box-shadow: 0px 0px 15px rgba(0, 0, 0, 0.4);">
         <strong>Bear Attacks in the US</strong>
     </div>
     '''
incident_map.get_root().html.add_child(folium.Element(title_html))

# Add a LayerControl to switch between different bear species layers
folium.LayerControl(collapsed=False).add_to(incident_map)

# Display the map in the Jupyter Notebook
display(incident_map)
